In [3]:
import parser
from pprint import pprint

from IPython.display import Image, display
from lark import Lark, Transformer
from lark import tree as larkTree
from lark.indenter import Indenter

import os

NOTES_FOLDER = './note_files/'
NOTE_PATHS = [x for x in os.listdir(NOTES_FOLDER) if x.endswith('.md')]

def display_tree(t, filename='viz.png'):
    larkTree.pydot__tree_to_png(t, filename)
    display(Image(filename))
    os.remove(filename)

In [4]:
for path in NOTE_PATHS:
    note_title = path.replace('.md', '')
    note_trees = parser.parse_note_tree(NOTES_FOLDER + path)
    print(note_title)
    for t in note_trees:
        display_tree(t)
    pprint(parser.parse_blocks(note_tree))

NameError: name 'NOTES_FOLDER' is not defined